# Datengruppierung in pandas
## pandas groupby

In [2]:
import pandas as pd

# Beispiel für das Importieren von CSV mit zahlreichen Anpassungen
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}  # es lassen sich nicht nur die Namen vorgeben, sondern auch die gewünschten Datentypen

df = pd.read_csv(
    "data/legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]  # auch Datumswerte können angepasst werden
)

df.head()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060 entries, 0 to 12059
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   last_name   12060 non-null  object        
 1   first_name  12060 non-null  category      
 2   birthday    11510 non-null  datetime64[ns]
 3   gender      12060 non-null  category      
 4   type        12060 non-null  category      
 5   state       12060 non-null  category      
 6   party       11828 non-null  category      
dtypes: category(5), datetime64[ns](1), object(1)
memory usage: 309.0+ KB


In [4]:
# probieren wir es mal aus...
df.groupby("state")

In [5]:
# Mmh, vielleicht auf eine Spalte zugreifen?
df.groupby("state")["last_name"]

In [6]:
# auch nicht - was können wir denn mit gruppierten Daten anfangen?
df.groupby("state")["last_name"].count()

state
AK      17
AL     209
AR     117
AS       2
AZ      49
CA     368
CO      92
CT     240
DC       2
DE      97
DK       9
FL     163
GA     317
GU       4
HI      24
IA     205
ID      59
IL     488
IN     344
KS     143
KY     373
LA     199
MA     427
MD     305
ME     175
MI     296
MN     162
MO     334
MS     155
MT      53
NC     356
ND      44
NE     128
NH     181
NJ     359
NM      57
NV      56
NY    1469
OH     676
OK      93
OL       2
OR      90
PA    1053
PI      13
PR      19
RI     107
SC     251
SD      51
TN     301
TX     264
UT      55
VA     434
VI       4
VT     115
WA      96
WI     198
WV     120
WY      40
Name: last_name, dtype: int64

Zur Erläuterung:
Der Methode _groupby_ wird die Spalte (oder die Spalten) übergeben, nach der gruppiert (angeordnet/aggregiert) werden soll, danach wird eine Spalte (oder mehrere) angegeben, auf der/denen die eigentliche Gruppierung durchgeführt werden soll.

In [13]:
df.groupby("gender").count()

,last_name,first_name,birthday,type,state,party
gender,,,,,,
F,249,249,249,249,249,249
M,11811,11811,11261,11811,11811,11579


wird keine Einschränkung angegeben, wird auf dem ganzen Dataframe gearbeitet

In [14]:
df.groupby(["state", "gender"]).last_name.count()

state  gender
AK     F           0
       M          17
AL     F           4
       M         205
AR     F           5
                ... 
WI     M         198
WV     F           1
       M         119
WY     F           1
       M          39
Name: last_name, Length: 116, dtype: int64

Wenn sie genau auf die Ausgabe schauen, sehen sie, dass _state_ und _gender_ hier Indizes sind (genauer: ein Multi-Index), keine eigenen Spalten.
Das kann gewünscht sein oder auch nicht, man kann es einstellen:

In [17]:
df.groupby(["state", "gender"], as_index=False)["last_name"].count()

,state,gender,last_name
0,AK,F,0
1,AK,M,17
2,AL,F,4
3,AL,M,205
4,AR,F,5
...,...,...,...
111,WI,M,198
112,WV,F,1
113,WV,M,119
114,WY,F,1


Groupby sortiert automatisch:

In [18]:
df.groupby(["state", "gender"], sort=False).last_name.count()

state  gender
DE     M          97
       F           0
VA     M         430
       F           4
SC     M         246
                ... 
VI     F           1
GU     M           3
       F           1
AS     M           2
       F           0
Name: last_name, Length: 116, dtype: int64

Wir haben jetzt immer gezählt. Aber was sind denn diese GroupBy-Objekte und warum kann man sie nicht anzeigen?

Hintergrund: lazy evaluation. Die Objekte werden erst angelegt, wenn sie wirklich gebraucht werden.
Wie war das noch bei yield-Generatoren? Einfach aufrufen:

In [19]:
for state, frame in df.groupby("state"):
    print(f"First 2 entries for {state!r}")
    print("------------------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
------------------------
     last_name first_name   birthday gender type state        party
6617    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6645      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
------------------------
    last_name first_name   birthday gender type state       party
911   Crowell       John 1780-09-18      M  rep    AL  Republican
990    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
------------------------
     last_name first_name   birthday gender type state party
1000     Bates      James 1788-08-25      M  rep    AR   NaN
1278    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
------------------------
          last_name first_name   birthday gender type state     party
10795         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11752  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

F

In [22]:
# welche Einträge gehören zu welcher Gruppe?
df.groupby("state").groups["CA"]  # California

Int64Index([ 2648,  2740,  2745,  2828,  2830,  2899,  3076,  3094,  3182,
             3219,
            ...
            11943, 11970, 11972, 11978, 11987, 12025, 12026, 12027, 12042,
            12049],
           dtype='int64', length=368)

In [24]:
# oder direkt auf die Gruppe zugreifen:
df.groupby("state").get_group("CA").head()

,last_name,first_name,birthday,gender,type,state,party
2648,Gilbert,Edward,NaT,M,rep,CA,Democrat
2740,Wright,George,1816-06-04,M,rep,CA,Independent
2745,Frémont,John,1813-01-21,M,sen,CA,Democrat
2828,Marshall,Edward,1821-06-29,M,rep,CA,Democrat
2830,McCorkle,Joseph,1819-06-24,M,rep,CA,Democrat


Beispiele teilweise übernommen aus:
https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
https://realpython.com/pandas-groupby/
Anpassungen HR

Datensatz US-Kongress
https://github.com/unitedstates/congress-legislators